In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import time
import tqdm
import scipy.stats as stats
import matplotlib.pyplot as plt

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

# API
from nats_bench import create

# custom modules
from custom.sss_model import get_cell_based_tiny_net
from ZeroShotProxy import *

2023-05-10 04:11:40.520746: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 04:11:40.565893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 04:11:41.315883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
parser = argparse.ArgumentParser("Training-free NAS on NATSBench (SSS)")
parser.add_argument("--data_path", type=str, default='./cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space", type=str, default='sss', help="The search space name.")
parser.add_argument("--config_path", type=str, default='./configs/nas-benchmark/algos/weight-sharing.config', help="The path to the configuration.")
parser.add_argument("--genotype", type=str, default="|nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|", help="The genotype.")
parser.add_argument("--affine", type=int, default=0, choices=[0, 1], help="Whether use affine=True or False in the BN layer.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
parser.add_argument("--algo", type=str, default="training-free", help="")


# log
parser.add_argument("--print_freq", type=int, default=200, help="print frequency (default: 200)")

# custom
parser.add_argument("--gpu", type=int, default=0, help="")
parser.add_argument("--workers", type=int, default=2, help="number of data loading workers")
parser.add_argument("--api_data_path", type=str, default="./api_data/NATS-sss-v1_0-50262-simple/", help="")
parser.add_argument("--save_dir", type=str, default='./results/tmp_sss', help="Folder to save checkpoints and log.")
parser.add_argument('--zero_shot_score', type=str, default='ETF62', help='could be: ETF; ZiCo (form ZiCo); params, (for \#Params); Zen (for Zen-NAS); TE (for TE-NAS)')
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])

if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

print(args.rand_seed)
print(args)
xargs=args

18875
Namespace(data_path='./cifar.python', dataset='cifar10', search_space='sss', config_path='./configs/nas-benchmark/algos/weight-sharing.config', genotype='|nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|', affine=0, track_running_stats=0, algo='training-free', print_freq=200, gpu=0, workers=2, api_data_path='./api_data/NATS-sss-v1_0-50262-simple/', save_dir='./results/tmp_sss', zero_shot_score='ETF62', rand_seed=18875)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=results/tmp_sss, use-tf=False, writer=None)
Arguments : -------------------------------
data_path        : ./cifar.python
dataset          : cifar10
search_space     : sss
config_path      : ./configs/nas-benchmark/algos/weight-sharing.config
genotype         : |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|
affine           : 0
track_running_stats : 0
algo             : training-free
print_freq       : 200
gpu              : 0
workers          : 2
api_data_path    : ./api_data/NATS-sss-v1_0-50262-simple/
save_dir         : ./results/tmp_sss
zero_shot_score  : ETF62
rand_seed        : 18875
Python  Version  : 3.10.9 (main, Mar  8 2023, 10:47:38) [GCC 11.2.0]
Pillow  Version  : 9.4.0
PyTorch Version  : 2.0.0
cuDNN   Version  : 8500
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
## API
api = create(xargs.api_data_path, xargs.search_space, fast_mode=True, verbose=False)
logger.log("Create API = {:} done".format(api))

## data
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, train_loader, valid_loader = get_nas_search_loaders(train_data,
                                                                   valid_data,
                                                                   xargs.dataset,
                                                                   "./configs/nas-benchmark/",
                                                                   (config.batch_size, config.test_batch_size),
                                                                   xargs.workers,)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

## model
search_space = get_search_spaces(xargs.search_space, "nats-bench")
model_config = dict2config(
        dict(
            name="generic",
            super_type="search-shape",
            candidate_Cs=search_space["candidates"],
            max_num_Cs=search_space["numbers"],
            num_classes=class_num,
            genotype=args.genotype,
            affine=bool(xargs.affine),
            track_running_stats=bool(xargs.track_running_stats),
        ),
        None,
    )
logger.log("search space : {:}".format(search_space))
logger.log("model config : {:}".format(model_config))
search_model = get_cell_based_tiny_net(model_config)
search_model.set_algo(xargs.algo)
logger.log("{:}".format(search_model))

device = torch.device('cuda:{}'.format(xargs.gpu))
network = search_model.to(device)

Create API = NATSsize(0/32768 architectures, fast_mode=True, file=) done
Files already downloaded and verified
Files already downloaded and verified
./configs/nas-benchmark/algos/weight-sharing.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=100, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=100, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
search space : {'candidates': [8, 16, 24, 32, 40, 48, 56, 64], 'numbers': 5}
model config : Configure(name='generic', super_type='search-shape', candidate_Cs=[8, 16, 24, 32, 40, 48, 56, 64], max_num_Cs=5, num_classes=10, genotype='|

In [5]:
def search_find_best(xargs, xloader, network, n_samples = None, archs = None):
    if 'etf' in xargs.zero_shot_score.lower():
        score_fn_name = "compute_{}_score".format(args.zero_shot_score.lower())
        score_fn = globals().get(score_fn_name)
        input_, target_ = next(iter(xloader))
        resolution = input_.size(2)
        batch_size = input_.size(0)
        zero_shot_score_dict = None
        arch_list = []
    else:
        raise NotImplementedError
    
    if archs is None and n_samples is not None:
        network.train()
        all_time = []
        all_mem = []
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        for i in tqdm.tqdm(range(n_samples)):
            # random sampling
            arch = network.random_genotype(True)

            start.record()
            torch.cuda.reset_peak_memory_stats()

            info_dict = score_fn.compute_nas_score(model=network, gpu=xargs.gpu, trainloader=None, resolution=resolution, batch_size=batch_size, search_space=xargs.search_space)

            end.record()
            torch.cuda.synchronize()
            all_time.append(start.elapsed_time(end))
            all_mem.append(torch.cuda.max_memory_reserved())
            torch.cuda.empty_cache()

            arch_list.append(arch)
            if zero_shot_score_dict is None: # initialize dict
                zero_shot_score_dict = dict()
                for k in info_dict.keys():
                    zero_shot_score_dict[k] = []
            for k, v in info_dict.items():
                zero_shot_score_dict[k].append(v)

        print("------Runtime------")
        print("All: {:.5f} ms".format(np.mean(all_time)))
        print("------Avg Mem------")
        print("All: {:.5f} GB".format(np.mean(all_mem)/1e9))
        print("------Max Mem------")
        print("All: {:.5f} GB".format(np.max(all_mem)/1e9))
        
    elif archs is not None and n_samples is None:
        network.train()
        all_time = []
        all_mem = []
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        for arch in tqdm.tqdm(archs):
            network.arch_cache = arch

            start.record()
            torch.cuda.reset_peak_memory_stats()

            info_dict = score_fn.compute_nas_score(model=network, gpu=xargs.gpu, trainloader=None, resolution=resolution, batch_size=batch_size, search_space=xargs.search_space)

            end.record()
            torch.cuda.synchronize()
            all_time.append(start.elapsed_time(end))
            all_mem.append(torch.cuda.max_memory_reserved())
            torch.cuda.empty_cache()

            arch_list.append(arch)
            if zero_shot_score_dict is None: # initialize dict
                zero_shot_score_dict = dict()
                for k in info_dict.keys():
                    zero_shot_score_dict[k] = []
            for k, v in info_dict.items():
                zero_shot_score_dict[k].append(v)

        print("------Runtime------")
        print("All: {:.5f} ms".format(np.mean(all_time)))
        print("------Avg Mem------")
        print("All: {:.5f} GB".format(np.mean(all_mem)/1e9))
        print("------Max Mem------")
        print("All: {:.5f} GB".format(np.max(all_mem)/1e9))
        
    return arch_list, zero_shot_score_dict

In [8]:
######### search across random N archs #########
archs, results = search_find_best(xargs, train_loader, network, n_samples=1)

# # search across N archs uniformly sampled according to test acc.
# def uniform_sample_archs(network, xargs, api, n_samples=1000, dataset='cifar10'):
#     arch = network.random_genotype(True)
#     search_space = get_search_spaces(xargs.search_space, "nats-bench")
#     archs = arch.gen_all(search_space, xargs.max_nodes, False)
    
#     def get_results_from_api(api, arch, dataset='cifar10'):
#         dataset_candidates = ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120']
#         assert dataset in dataset_candidates
#         index = api.query_index_by_arch(arch)
#         api._prepare_info(index)
#         archresult = api.arch2infos_dict[index]['200']
#         if dataset == 'cifar10-valid':
#             acc = archresult.get_metrics(dataset, 'x-valid', iepoch=199, is_random=False)['accuracy']
#         else:
#             acc = archresult.get_metrics(dataset, 'ori-test', iepoch=199, is_random=False)['accuracy']
#         return acc

#     accs = []
#     for a in archs:
#         accs.append(get_results_from_api(api, a, dataset))
#     interval = len(archs) // n_samples
#     sorted_indices = np.argsort(accs)
#     new_archs = []
#     for i, idx in enumerate(sorted_indices):
#         if i % interval == 0:
#             new_archs.append(archs[idx])
#     archs = new_archs
    
#     return archs

# uniform_archs = uniform_sample_archs(network, xargs, api, 1000, 'cifar10')
# archs, results = search_find_best(xargs, train_loader, network, archs=uniform_archs)


# ######### search across all archs #########
# def generate_all_archs(network, xargs):
#     arch = network.random_genotype(True)
#     search_space = get_search_spaces(xargs.search_space, "nats-bench")
#     archs = arch.gen_all(search_space, xargs.max_nodes, False)
#     return archs

# all_archs = generate_all_archs(network, xargs)
# archs, results = search_find_best(xargs, train_loader, network, archs=all_archs)

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([64, 64, 8, 8])
torch.Size([64, 64, 8, 8])
torch.Size([64, 8, 8, 8])
torch.Size([64, 64, 8, 8])
tensor([-1., -1., -1.,  1., -1.,  1., -1., -1.], device='cuda:0')
tensor([-1.3958, -1.0000, -2.2439,  1.0000, -0.4246,  1.0000, -1.2408, -1.0000,
        -0.4609,  0.0000, -0.9131, -0.6275,  0.0000,  0.0000, -0.3700,  0.7735,
         0.0000,  0.0000, -0.0931,  0.4326,  0.0000,  0.3458,  0.0000,  0.0829,
         0.0986,  0.0000,  0.0000,  0.0000,  0.6672,  0.0000, -1.3828,  0.1540,
         0.0000, -0.5559,  1.4028,  0.0000,  0.0000, -0.7136, -0.4147, -0.4712,
         0.0000,  0.0000,  0.7122,  0.0000,  0.3337,  0.0000,  0.0000,  0.0000,
         0.6368,  0.0000,  0.5347,  0.2823,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0420, -0.2295],
       device='cuda:0')

torch.Size([64, 64, 8, 8])
torch.Size([64, 64, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 40, 16, 16])
tensor([ 1.,  1., -1., -1.,  1.,  1., -1.,  1.,  

  0%|          | 0/1 [00:38<?, ?it/s]


KeyboardInterrupt: Interrupted by user

In [ ]:
# api.get_more_info(1,'cifar10')
# api.query_by_index(1)
# archresult = api.arch2infos_dict[1]['90']
# archresult.get_metrics('cifar10', 'ori-test', iepoch=89, is_random=False)['accuracy']
# archresult.get_metrics('cifar100', 'ori-test', iepoch=89, is_random=False)['accuracy']
# archresult.get_metrics('ImageNet16-120', 'ori-test', iepoch=89, is_random=False)['accuracy']

def get_results_from_api(api, arch, dataset='cifar10'):
    dataset_candidates = ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120']
    assert dataset in dataset_candidates
    index = api.query_index_by_arch(arch)
    api._prepare_info(index)
    archresult = api.arch2infos_dict[index]['90']
    
    if dataset == 'cifar10-valid':
        acc = archresult.get_metrics(dataset, 'x-valid', iepoch=89, is_random=False)['accuracy']
    else:
        acc = archresult.get_metrics(dataset, 'ori-test', iepoch=89, is_random=False)['accuracy']
    flops = archresult.get_compute_costs(dataset)['flops']
    params = archresult.get_compute_costs(dataset)['params']
    
    return acc, flops, params

api_valid_accs, api_flops, api_params = [], [], []
for a in archs:
#     valid_acc, flops, params = get_results_from_api(api, a, 'cifar10')
#     valid_acc, flops, params = get_results_from_api(api, a, 'cifar100')
    valid_acc, flops, params = get_results_from_api(api, a, 'ImageNet16-120')
    api_valid_accs.append(valid_acc)
    api_flops.append(flops)
    api_params.append(params)

In [ ]:
fig_scale = 1.1

l = len(api_flops)
rank_agg = np.log(stats.rankdata(api_flops) / l)
for k in results.keys():
    rank_agg = rank_agg + np.log( stats.rankdata(results[k]) / l)
    
# rank_agg = None
# for k in results.keys():
#     if rank_agg is None:
#         rank_agg = np.log( stats.rankdata(results[k]) / l)
#     else:
#         rank_agg = rank_agg + np.log( stats.rankdata(results[k]) / l)
    
best_idx = np.argmax(rank_agg)

best_arch, acc = archs[best_idx], api_valid_accs[best_idx]
if api is not None:
    print("{:}".format(api.query_by_arch(best_arch, "90")))
    

x = stats.rankdata(rank_agg)
y = stats.rankdata(api_valid_accs)
kendalltau = stats.kendalltau(x, y)
spearmanr = stats.spearmanr(x, y)
pearsonr = stats.pearsonr(x, y)
print("aggregated: {}\t{}\t{}\t".format(kendalltau[0], pearsonr[0], spearmanr[0]))
plt.figure(figsize=(4*fig_scale,3*fig_scale))
plt.scatter(x, y, linewidths=0.1)
plt.scatter(x[best_idx], y[best_idx], c="r", linewidths=0.1)
plt.title("rank_agg")
plt.show()


# for each
tau, p_value = stats.kendalltau(stats.rankdata(api_flops), stats.rankdata(api_valid_accs))
x = stats.rankdata(api_flops)
y = stats.rankdata(api_valid_accs)
kendalltau = stats.kendalltau(x, y)
spearmanr = stats.spearmanr(x, y)
pearsonr = stats.pearsonr(x, y)
print("FLOPs: {}\t{}\t{}\t".format(kendalltau[0], pearsonr[0], spearmanr[0]))
plt.figure(figsize=(4*fig_scale,3*fig_scale))
plt.scatter(x, y, linewidths=0.1)
plt.scatter(x[best_idx], y[best_idx], c="r", linewidths=0.1)
plt.title("rank_FLOPs")
plt.show()
for k in results.keys():
    x = stats.rankdata(results[k])
    y = stats.rankdata(api_valid_accs)
    kendalltau = stats.kendalltau(x, y)
    spearmanr = stats.spearmanr(x, y)
    pearsonr = stats.pearsonr(x, y)
    print("{}: {}\t{}\t{}\t".format(k, kendalltau[0], pearsonr[0], spearmanr[0]))
    plt.figure(figsize=(4*fig_scale,3*fig_scale))
    plt.scatter(x, y, linewidths=0.1)
    plt.scatter(x[best_idx], y[best_idx], c="r", linewidths=0.1)
    plt.title("rank_{}".format(k))
    plt.show()